# Importing all the good stuff

In [1]:
import numpy as np
import pandapower as pp
import pandapower.networks as pn
#import pandapower.converter
#import pandapower.estimation
#import pandapower.topology
#import pandapower.plotting
#import inspect
#import pandapower.test
#pandapower.test.run_all_tests()

# Load a case and print its parameters

In [2]:
net = pn.case14()
print(net)
net.line

This pandapower network includes the following parameter tables:
   - bus (14 elements)
   - load (11 elements)
   - gen (4 elements)
   - shunt (1 element)
   - ext_grid (1 element)
   - line (15 elements)
   - trafo (5 elements)
   - poly_cost (5 elements)
   - bus_geodata (14 elements)
 and the following results tables:
   - res_line (15 elements)
   - res_trafo (5 elements)
   - res_ext_grid (1 element)
   - res_load (11 elements)
   - res_shunt (1 element)
   - res_gen (4 elements)


,c_nf_per_km,df,from_bus,g_us_per_km,in_service,length_km,max_i_ka,max_loading_percent,name,parallel,r_ohm_per_km,std_type,to_bus,type,x_ohm_per_km
0,768.484773,1.0,0,0.0,True,1.0,42.339020,100.0,None,1,3.532005,None,1,ol,10.783732
1,716.088084,1.0,0,0.0,True,1.0,42.339020,100.0,None,1,9.846967,None,4,ol,40.649040
10,0.000000,1.0,8,0.0,True,1.0,27479.652235,100.0,None,1,0.000014,None,9,ol,0.000037
11,0.000000,1.0,8,0.0,True,1.0,27479.652235,100.0,None,1,0.000055,None,13,ol,0.000117
12,0.000000,1.0,9,0.0,True,1.0,27479.652235,100.0,None,1,0.000035,None,10,ol,0.000083
13,0.000000,1.0,11,0.0,True,1.0,27479.652235,100.0,None,1,0.000096,None,12,ol,0.000086
14,0.000000,1.0,12,0.0,True,1.0,27479.652235,100.0,None,1,0.000074,None,13,ol,0.000151
2,637.493051,1.0,1,0.0,True,1.0,42.339020,100.0,None,1,8.563928,None,2,ol,36.080033
3,494.857619,1.0,1,0.0,True,1.0,42.339020,100.0,None,1,10.590547,None,3,ol,32.134320
4,503.590401,1.0,1,0.0,True,1.0,42.339020,100.0,None,1,10.379138,None,4,ol,31.689630


# Extract needed information to build Ybus matrix

In [3]:
#net.line[["from_bus","to_bus","length_km","r_ohm_per_km","x_ohm_per_km"]]
nbus = len(net.bus) # Number of buses in the system
r = net.line["length_km"]*net.line["r_ohm_per_km"] # Total resistance of lines
x = net.line["length_km"]*net.line["x_ohm_per_km"] # Total reactance of lines
z = r.values+x.values*1j # Total resistance of lines
from_buses = net.line["from_bus"].values # "From bus" vector
to_buses = net.line["to_bus"].values  # "To bus" vector
Zb=(135000**2)/(100*10**6)
Zpu = z/Zb

## Calculating Yij and Yjj elements seperately

In [4]:
# From definition:  the off diagonal entries are the negatives of the admittance
#                   of lines between buses i and j. If there is no line between i
#                   and j this term is zero
yij = 1/Zpu


# From definition:  the diagonal entries of Y bus are found by summing the primitive 
#                   admittance of lines lines and ties to the reference at bus j.

bus_connections = np.concatenate([from_buses, to_buses])
y_connections = np.concatenate([yij,yij])
yjj = np.zeros(nbus,dtype=np.complex_)
for i in range(0,nbus):
    yjj[i] = sum(y_connections[np.where((bus_connections == i))])


In [5]:
Ybus = np.zeros((nbus,nbus),dtype=np.complex_)
for i in range(0,nbus):
    Ybus[from_buses[i]][to_buses[i]] = -yij[i]
    Ybus[to_buses[i]][from_buses[i]] = -yij[i]
    Ybus[i][i]                       = yjj[i]


In [6]:
Ybus

array([[ 6.02502906e+00-1.94980702e+01j, -4.99913160e+00+1.52630865e+01j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        -1.02589745e+00+4.23498368e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j],
       [-4.99913160e+00+1.52630865e+01j,  9.52132361e+00-3.03547154e+01j,
        -1.13501919e+00+4.78186315e+00j, -1.68603315e+00+5.11583833e+00j,
        -1.70113967e+00+5.19392740e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+0

In [9]:
net.load

,bus,const_i_percent,const_z_percent,controllable,in_service,name,p_mw,q_mvar,scaling,sn_mva,type
0,1,0.0,0.0,False,True,None,21.7,12.7,1.0,NaN,None
1,2,0.0,0.0,False,True,None,94.2,19.0,1.0,NaN,None
10,13,0.0,0.0,False,True,None,14.9,5.0,1.0,NaN,None
2,3,0.0,0.0,False,True,None,47.8,-3.9,1.0,NaN,None
3,4,0.0,0.0,False,True,None,7.6,1.6,1.0,NaN,None
4,5,0.0,0.0,False,True,None,11.2,7.5,1.0,NaN,None
5,8,0.0,0.0,False,True,None,29.5,16.6,1.0,NaN,None
6,9,0.0,0.0,False,True,None,9.0,5.8,1.0,NaN,None
7,10,0.0,0.0,False,True,None,3.5,1.8,1.0,NaN,None
8,11,0.0,0.0,False,True,None,6.1,1.6,1.0,NaN,None


In [10]:
net.gen

,bus,controllable,in_service,name,p_mw,scaling,sn_mva,type,vm_pu,slack,max_p_mw,min_p_mw,max_q_mvar,min_q_mvar
0,1,True,True,None,40.0,1.0,NaN,None,1.045,False,140.0,0.0,50.0,-40.0
1,2,True,True,None,0.0,1.0,NaN,None,1.010,False,100.0,0.0,40.0,0.0
2,5,True,True,None,0.0,1.0,NaN,None,1.070,False,100.0,0.0,24.0,-6.0
3,7,True,True,None,0.0,1.0,NaN,None,1.090,False,100.0,0.0,24.0,-6.0


In [11]:
net.shunt

,bus,name,q_mvar,p_mw,vn_kv,step,max_step,in_service
0,8,None,-19.0,0.0,0.208,1,1,True


In [12]:
net.trafo

,df,hv_bus,i0_percent,in_service,lv_bus,max_loading_percent,name,parallel,pfe_kw,shift_degree,...,tap_min,tap_phase_shifter,tap_pos,tap_side,tap_step_degree,tap_step_percent,vn_hv_kv,vn_lv_kv,vk_percent,vkr_percent
0,1.0,3,0.0,True,6,100.0,None,1,0.0,0.0,...,NaN,False,-1.0,hv,NaN,2.2,135.0,14.000,2070.288,0.0
1,1.0,3,0.0,True,8,100.0,None,1,0.0,0.0,...,NaN,False,-1.0,hv,NaN,3.1,135.0,0.208,5506.182,0.0
2,1.0,4,0.0,True,5,100.0,None,1,0.0,0.0,...,NaN,False,-1.0,hv,NaN,6.8,135.0,0.208,2494.998,0.0
3,1.0,6,0.0,True,7,100.0,None,1,0.0,0.0,...,NaN,False,NaN,None,NaN,NaN,14.0,12.000,1743.885,0.0
4,1.0,6,0.0,True,8,100.0,None,1,0.0,0.0,...,NaN,False,NaN,None,NaN,NaN,14.0,0.208,1089.099,0.0


In [13]:
net.poly_cost

,element,et,cp0_eur,cp1_eur_per_mw,cp2_eur_per_mw2,cq0_eur,cq1_eur_per_mvar,cq2_eur_per_mvar2
0,0,ext_grid,0.0,-20.0,-0.043029,0.0,0.0,0.0
1,0,gen,0.0,-20.0,-0.250000,0.0,0.0,0.0
2,1,gen,0.0,-40.0,-0.010000,0.0,0.0,0.0
3,2,gen,0.0,-40.0,-0.010000,0.0,0.0,0.0
4,3,gen,0.0,-40.0,-0.010000,0.0,0.0,0.0


In [14]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
net.bus

,in_service,max_vm_pu,min_vm_pu,name,type,vn_kv,zone
0,True,1.06,0.94,1,b,135.000,1
1,True,1.06,0.94,2,b,135.000,1
10,True,1.06,0.94,11,b,0.208,1
11,True,1.06,0.94,12,b,0.208,1
12,True,1.06,0.94,13,b,0.208,1
13,True,1.06,0.94,14,b,0.208,1
2,True,1.06,0.94,3,b,135.000,1
3,True,1.06,0.94,4,b,135.000,1
4,True,1.06,0.94,5,b,135.000,1
5,True,1.06,0.94,6,b,0.208,1
